In [ ]:
import itertools
import numpy as np
import os
import seaborn as sns
from tqdm import tqdm
from dataclasses import asdict, dataclass, field
import vsketch
import shapely.geometry as sg
from shapely.geometry import box, MultiLineString, Point, MultiPoint, Polygon, MultiPolygon, LineString
import shapely.affinity as sa
import shapely.ops as so
import matplotlib.pyplot as plt
import pandas as pd

import vpype_cli
from typing import List, Generic
from genpen import genpen as gp, utils as utils
from scipy import stats as ss
import geopandas
from shapely.errors import TopologicalError
import functools
%load_ext autoreload
%autoreload 2
import vpype
from skimage import io
from pathlib import Path

import bezier

from sklearn.preprocessing import minmax_scale
from skimage import feature
from genpen.utils import Paper
from genpen.genpen import *

In [ ]:
# make page
paper_size = '11x14 inches'
border:float=30
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
layers = []

In [ ]:
sk.document.bounds()

In [ ]:
import vpype as vp

In [ ]:
vp.convert_page_size('11x14in')

In [ ]:
xgen = ss.uniform(loc=0.6, scale=0.2).rvs
split_func = functools.partial(split_along_longest_side_of_min_rectangle, xgen=xgen)
splits = recursive_split_frac_buffer(
    drawbox, 
    split_func=split_func,
    p_continue=0.85, 
    depth=0, 
    depth_limit=3,
    buffer_frac=-0.0
)

bps = MultiPolygon([p for p in splits])

   
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.5mm')
sk.geometry(bps.boundary)

# tolerance=0.5

sk.display()

In [ ]:
n_layers = 1

In [ ]:

for ii in range(n_layers):
    fills = []
    for p in bps:
        xjitter_func = 0
        yjitter_func = ss.norm(loc=0, scale=np.random.uniform(0.01, 0.3)).rvs
        bhf = BezierHatchFill(
            spacing=np.random.uniform(0.2, 0.35),
            degrees=np.random.uniform(40,60),
            poly_to_fill=p, 
            xjitter_func=xjitter_func, 
            yjitter_func=yjitter_func,
            fill_inscribe_buffer=1.4,
            n_nodes_per_line=15,
            n_eval_points=100,
        )
        fills.append(bhf.p)

    fills = [f for f in fills if f.length > 0]
    layer = gp.merge_LineStrings(fills)
    layers.append(layer)

In [ ]:
xgen = ss.uniform(loc=0.6, scale=0.02).rvs
split_func = functools.partial(split_along_longest_side_of_min_rectangle, xgen=xgen)
splits = recursive_split_frac_buffer(
    drawbox, 
    split_func=split_func,
    p_continue=0.85, 
    depth=0, 
    depth_limit=3,
    buffer_frac=-0.0
)

bps = MultiPolygon([p for p in splits])

   
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.5mm')
sk.geometry(bps.boundary)

# tolerance=0.5

sk.display()

In [ ]:
n_layers = 1

In [ ]:

for ii in range(n_layers):
    fills = []
    for p in bps:
        xjitter_func = 0
        yjitter_func = ss.norm(loc=0, scale=np.random.uniform(0.01, 0.3)).rvs
        bhf = BezierHatchFill(
            spacing=np.random.uniform(0.2, 0.35),
            degrees=np.random.uniform(40,60),
            poly_to_fill=p, 
            xjitter_func=xjitter_func, 
            yjitter_func=yjitter_func,
            fill_inscribe_buffer=1.4,
            n_nodes_per_line=15,
            n_eval_points=100,
        )
        fills.append(bhf.p)

    fills = [f for f in fills if f.length > 0]
    layer = gp.merge_LineStrings(fills)
    layers.append(layer)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)

# sk.penWidth('0.2')
# for tolerance in [0.1, 0.3, 1.5]:
#     sk.vpype(f'linemerge --tolerance {tolerance}mm')
# sk.vpype('linesimplify --tolerance 0.1 linesort')

sk.display(color_mode='layer')

In [ ]:
sk.save('/home/naka/art/plotter_svgs/0002_subdivide_bez_shading.svg')

simpler, draw bezs one by one

In [ ]:
# make page
paper_size = '11x14 inches'
border:float=30
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
db = Shape(drawbox)

In [ ]:
yspacing = 0.2

In [ ]:
ystarts = np.arange(db.bottom, db.top, yspacing)

In [ ]:
xjitter_func = 0.
yjitter_func = ss.norm(loc=0, scale=np.random.uniform(0.01, 0.3)).rvs
bhf = BezierHatchFill(
    spacing=0.4,
    degrees=0,
    poly_to_fill=drawbox, 
    xjitter_func=xjitter_func, 
    yjitter_func=yjitter_func,
    fill_inscribe_buffer=1.4,
    n_nodes_per_line=20,
    n_eval_points=100,
    alternate_direction=False
)

In [ ]:
bhf.lines[0].length

In [ ]:
lines = bhf.lines.intersection(drawbox)

In [ ]:
lines = MultiLineString([l for l in lines if l.length > 200])

In [ ]:
n_lines = len(lines)

In [ ]:
eval_starts = gaussian_random_walk(n=n_lines, step_init=0.05, step_mu=0.01, step_std=0.5, scale=True)
eval_ends = gaussian_random_walk(n=n_lines, step_init=0.95, step_mu=-0.01, step_std=0.5, scale=True)

In [ ]:
clipped_lines = []
for ii,(line, start, end) in enumerate(zip(lines, eval_starts, eval_ends)):
    eval_points = np.linspace(start, end, n_eval_points)
    pts = [line.interpolate(p, normalized=True) for p in eval_points]
    if ii % 2 == 0:
        pts = list(reversed(pts))
    clipped_line = LineString(pts)
    clipped_lines.append(clipped_line)
    
clines = MultiLineString(clipped_lines)

In [ ]:
layers = [clines]

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)

# sk.penWidth('0.2')
# for tolerance in [0.1, 0.3, 1.5]:
#     sk.vpype(f'linemerge --tolerance {tolerance}mm')
# sk.vpype('linesimplify --tolerance 0.1 linesort')

sk.display(color_mode='layer')

In [ ]:
sk.save('/home/naka/art/plotter_svgs/0001_bez_shading.svg')

# more overlappers

In [ ]:
# make page
paper_size = '11x14 inches'
border:float=35
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
layers = []

In [ ]:
xgen = ss.uniform(loc=0.4, scale=0.1).rvs
split_func = functools.partial(split_along_longest_side_of_min_rectangle, xgen=xgen)
splits = recursive_split_frac_buffer(
    drawbox, 
    split_func=split_func,
    p_continue=0.85, 
    depth=0, 
    depth_limit=3,
    buffer_frac=-0.0
)

bps = MultiPolygon([p for p in splits])

   
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.5mm')
sk.geometry(bps.boundary)

# tolerance=0.5

sk.display()

In [ ]:
n_layers = 1

In [ ]:

for ii in range(n_layers):
    fills = []
    for p in bps:
        xjitter_func = 0
        yjitter_func = ss.norm(loc=0, scale=np.random.uniform(0.01, 0.6)).rvs
        bhf = BezierHatchFill(
            spacing=np.random.uniform(0.2, 0.35),
            degrees=np.random.uniform(10,90),
            poly_to_fill=p, 
            xjitter_func=xjitter_func, 
            yjitter_func=yjitter_func,
            fill_inscribe_buffer=1.4,
            n_nodes_per_line=15,
            n_eval_points=100,
        )
        fills.append(bhf.p)

    fills = [f for f in fills if f.length > 0]
    layer = gp.merge_LineStrings(fills)
    layers.append(layer)

In [ ]:
xgen = ss.uniform(loc=0.6, scale=0.02).rvs

split_func = functools.partial(random_bezier_subdivide, x0=0.15, x1=0.85, n_eval_points=50)
splits = recursive_split_frac_buffer(
    drawbox, 
    split_func=split_func,
    p_continue=0.85, 
    depth=0, 
    depth_limit=4,
    buffer_frac=-0.0
)

bps = MultiPolygon([p for p in splits])

   
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.5mm')
sk.geometry(bps.boundary)

# tolerance=0.5

sk.display()

In [ ]:
n_layers = 1

In [ ]:

for ii in range(n_layers):
    fills = []
    for p in bps:
        xjitter_func = 0
        yjitter_func = ss.norm(loc=0, scale=np.random.uniform(0.01, 0.7)).rvs
        bhf = BezierHatchFill(
            spacing=np.random.uniform(0.2, 0.35),
            degrees=np.random.uniform(40,60),
            poly_to_fill=p, 
            xjitter_func=xjitter_func, 
            yjitter_func=yjitter_func,
            fill_inscribe_buffer=1.4,
            n_nodes_per_line=15,
            n_eval_points=100,
        )
        fills.append(bhf.p)

    fills = [f for f in fills if f.length > 0]
    layer = gp.merge_LineStrings(fills)
    layers.append(layer)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)

# sk.penWidth('0.2')
# for tolerance in [0.1, 0.3, 1.5]:
#     sk.vpype(f'linemerge --tolerance {tolerance}mm')
# sk.vpype('linesimplify --tolerance 0.1 linesort')

sk.display(color_mode='layer')

In [ ]:
sk.save('/home/naka/art/plotter_svgs/0003_subdivide_bez_shading.svg')

In [ ]:
# make page
paper_size = '14x11 inches'
border:float=35
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
layers = []

In [ ]:
xgen = ss.uniform(loc=0.3, scale=0.1).rvs
split_func = functools.partial(split_along_longest_side_of_min_rectangle, xgen=xgen)
splits = recursive_split_frac_buffer(
    drawbox, 
    split_func=split_func,
    p_continue=0.7, 
    depth=0, 
    depth_limit=5,
    buffer_frac=-0.0
)

bps = MultiPolygon([p for p in splits])

   
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.5mm')
sk.geometry(bps.boundary)

# tolerance=0.5

sk.display()

In [ ]:
n_layers = 1

In [ ]:

for ii in range(n_layers):
    fills = []
    for p in bps:
        xjitter_func = 0
        yjitter_func = ss.norm(loc=0, scale=np.random.uniform(0.01, 0.4)).rvs
        bhf = BezierHatchFill(
            spacing=np.random.uniform(0.18, 0.35),
            degrees=np.random.uniform(35,55),
            poly_to_fill=p, 
            xjitter_func=xjitter_func, 
            yjitter_func=yjitter_func,
            fill_inscribe_buffer=1.4,
            n_nodes_per_line=15,
            n_eval_points=100,
        )
        fills.append(bhf.p)

    fills = [f for f in fills if f.length > 0]
    layer = gp.merge_LineStrings(fills)
    layers.append(layer)

In [ ]:
xgen = ss.uniform(loc=0.3, scale=0.1).rvs
split_func = functools.partial(split_along_longest_side_of_min_rectangle, xgen=xgen)
splits = recursive_split_frac_buffer(
    drawbox, 
    split_func=split_func,
    p_continue=0.7, 
    depth=0, 
    depth_limit=5,
    buffer_frac=-0.0
)

bps = MultiPolygon([p for p in splits])

   
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.5mm')
sk.geometry(bps.boundary)

# tolerance=0.5

sk.display()

In [ ]:

for ii in range(n_layers):
    fills = []
    for p in bps:
        xjitter_func = 0
        yjitter_func = ss.norm(loc=0, scale=np.random.uniform(0.01, 0.4)).rvs
        bhf = BezierHatchFill(
            spacing=np.random.uniform(0.2, 0.35),
            degrees=np.random.uniform(35,55),
            poly_to_fill=p, 
            xjitter_func=xjitter_func, 
            yjitter_func=yjitter_func,
            fill_inscribe_buffer=1.4,
            n_nodes_per_line=15,
            n_eval_points=100,
        )
        fills.append(bhf.p)

    fills = [f for f in fills if f.length > 0]
    layer = gp.merge_LineStrings(fills)
    layers.append(layer)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)

# sk.penWidth('0.2')
# for tolerance in [0.1, 0.3, 1.5]:
#     sk.vpype(f'linemerge --tolerance {tolerance}mm')
# sk.vpype('linesimplify --tolerance 0.1 linesort')

sk.display(color_mode='layer')

In [ ]:
sk.save('/home/naka/art/plotter_svgs/0004_subdivide_bez_shading.svg')

In [ ]:
# make page
paper_size = '14x11 inches'
border:float=35
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
layers = []

In [ ]:
xgen = ss.uniform(loc=0.6, scale=0.02).rvs

split_func = functools.partial(random_bezier_subdivide, x0=0.5, x1=0.85, n_eval_points=50)
splits = recursive_split_frac_buffer(
    drawbox, 
    split_func=split_func,
    p_continue=0.85, 
    depth=0, 
    depth_limit=2,
    buffer_frac=-0.0
)

bps = MultiPolygon([p for p in splits])

   
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.5mm')
sk.geometry(bps.boundary)

# tolerance=0.5

sk.display()

In [ ]:
n_layers = 1

In [ ]:

for ii in range(n_layers):
    fills = []
    for p in bps:
        xjitter_func = 0
        yjitter_func = ss.norm(loc=0, scale=np.random.uniform(0.01, 0.9)).rvs
        bhf = BezierHatchFill(
            spacing=np.random.uniform(0.2, 0.35),
            degrees=30,
            poly_to_fill=p, 
            xjitter_func=xjitter_func, 
            yjitter_func=yjitter_func,
            fill_inscribe_buffer=1.4,
            n_nodes_per_line=15,
            n_eval_points=100,
        )
        fills.append(bhf.p)

    fills = [f for f in fills if f.length > 0]
    layer = gp.merge_LineStrings(fills)
    layers.append(layer)

In [ ]:
xgen = ss.uniform(loc=0.7, scale=0.02).rvs

split_func = functools.partial(random_bezier_subdivide, x0=0.15, x1=0.85, n_eval_points=50)
splits = recursive_split_frac_buffer(
    drawbox, 
    split_func=split_func,
    p_continue=0.85, 
    depth=0, 
    depth_limit=2,
    buffer_frac=-0.0
)

bps = MultiPolygon([p for p in splits])

   
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.5mm')
sk.geometry(bps.boundary)

# tolerance=0.5

sk.display()

In [ ]:
n_layers = 1

In [ ]:

for ii in range(n_layers):
    fills = []
    for p in bps:
        xjitter_func = 0
        yjitter_func = ss.norm(loc=0, scale=np.random.uniform(0.01, 0.6)).rvs
        bhf = BezierHatchFill(
            spacing=np.random.uniform(0.2, 0.35),
            degrees=30,
            poly_to_fill=p, 
            xjitter_func=xjitter_func, 
            yjitter_func=yjitter_func,
            fill_inscribe_buffer=1.4,
            n_nodes_per_line=15,
            n_eval_points=100,
        )
        fills.append(bhf.p)

    fills = [f for f in fills if f.length > 0]
    layer = gp.merge_LineStrings(fills)
    layers.append(layer)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)

# sk.penWidth('0.2')
# for tolerance in [0.1, 0.3, 1.5]:
#     sk.vpype(f'linemerge --tolerance {tolerance}mm')
# sk.vpype('linesimplify --tolerance 0.1 linesort')

sk.display(color_mode='layer')

In [ ]:
sk.save('/home/naka/art/plotter_svgs/0005_subdivide_bez_shading.svg')